User Input → Preprocessing → Prompt → LLM → Response → History Update (pipeline workflow)

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "add your api key here"

In [2]:
# Import required libraries
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.history import RunnableWithMessageHistory

In [3]:
# Step 2: Define Cafe Menu and Hit Dish
cafe_menu = {
    "Espresso": {"price": 3.00, "tags": ["coffee", "quick"], "ingredients": "espresso beans, water", "description": "A bold, quick caffeine kick!"},
    "Latte": {"price": 4.00, "tags": ["coffee", "creamy"], "ingredients": "espresso, steamed milk", "description": "Smooth and creamy coffee bliss."},
    "Muffin": {"price": 2.50, "tags": ["sweet", "snack"], "ingredients": "flour, sugar, blueberries", "description": "Sweet blueberry goodness in every bite."},
    "Avocado Toast": {"price": 5.00, "tags": ["trendy", "healthy"], "ingredients": "avocado, toast, lemon", "description": "Trendy, fresh, and oh-so-tasty!", "hit": True}
}
hit_dish = "Avocado Toast"

In [4]:
# Step 3: Initialize LLM
llm = ChatGroq(model="llama3-70b-8192", temperature=0)

In [5]:
# Step 4: Chat History and Cart Management
chat_histories = {}
carts = {}

In [6]:
def get_chat_history(session_id: str):
    if session_id not in chat_histories:
        chat_histories[session_id] = InMemoryChatMessageHistory()
    return chat_histories[session_id]

In [7]:
def get_cart(session_id: str):
    if session_id not in carts:
        carts[session_id] = []
    return carts[session_id]

In [8]:
# Step 5: Dynamic Menu Filtering Function
def filter_menu(user_input, menu):
    user_input = user_input.lower()
    filtered_items = []

    # Price-based queries
    if "under" in user_input or "cheap" in user_input:
        try:
            price_limit = float(next(word for word in user_input.split() if word.replace("$", "").replace(".", "").isdigit()).replace("$", ""))
            filtered_items = [item for item, data in menu.items() if data["price"] <= price_limit]
        except (StopIteration, ValueError):
            filtered_items = [item for item, data in menu.items() if data["price"] <= 4.00]  # Default to $4

    # Tag-based queries
    elif any(tag in user_input for tag in ["sweet", "coffee", "creamy", "healthy", "quick", "trendy", "snack"]):
        for tag in user_input.split():
            filtered_items.extend([item for item, data in menu.items() if tag in data["tags"]])
        filtered_items = list(set(filtered_items))  # Remove duplicates

    # Default: return all items if no specific query detected
    if not filtered_items:
        filtered_items = list(menu.keys())

    # Format with rich descriptions
    result = "\n".join(f"{item}: ${menu[item]['price']:.2f} - {menu[item]['description']} ({', '.join(menu[item]['tags'])})" 
                      for item in filtered_items)
    return result if result else "Sorry, nothing matches that! Try our hit dish, Avocado Toast 🥑!"

In [9]:
# Step 6: Order System Function
def handle_order(user_input, menu, cart):
    user_input = user_input.lower()
    if "take" in user_input or "order" in user_input or "want" in user_input:
        for item in menu:
            if item.lower() in user_input:
                cart.append(item)
                return f"Added {item} to your cart! 🛒 Anything else? (Cart: {', '.join(cart)})"
        return "Sorry, we don’t have that! Check the menu or try our hit dish, Avocado Toast 🥑."
    elif "cart" in user_input:
        return f"Your cart: {', '.join(cart) if cart else 'empty'} 🛒. Ready to order more?"
    elif "clear cart" in user_input:
        cart.clear()
        return "Cart cleared! Start fresh? ✨"
    return None

In [10]:
# Step 7: Preprocessing Function for Chain
def preprocess_input(input_dict, config=None):
    user_input = input_dict["user_input"]
    session_id = input_dict.get("session_id", "user1")  # Default session ID
    cart = get_cart(session_id)
    
    # Get chat history from the session
    chat_history = get_chat_history(session_id).messages if session_id in chat_histories else []

    # Handle order-related input first
    order_response = handle_order(user_input, cafe_menu, cart)
    if order_response:
        return {
            "user_input": user_input,
            "menu": "\n".join(f"{item}: ${data['price']:.2f} - {data['description']} ({', '.join(data['tags'])})" 
                              for item, data in cafe_menu.items()),
            "hit_dish": hit_dish,
            "filtered_menu": order_response,
            "cart": ", ".join(cart) if cart else "empty",
            "chat_history": chat_history  # Pass chat history explicitly
        }

    # Otherwise, filter menu dynamically
    full_menu_str = "\n".join(f"{item}: ${data['price']:.2f} - {data['description']} ({', '.join(data['tags'])})" 
                              for item, data in cafe_menu.items())
    filtered_menu = filter_menu(user_input, cafe_menu)
    
    return {
        "user_input": user_input,
        "menu": full_menu_str,
        "hit_dish": hit_dish,
        "filtered_menu": filtered_menu,
        "cart": ", ".join(cart) if cart else "empty",
        "chat_history": chat_history  # Pass chat history explicitly
    }

In [11]:
# Step 8: Updated Prompt with Rich Responses and Order Support
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are CafeSnap, a friendly cafe chatbot ☕. Help users with the menu. Full menu:\n{menu}\nThe hit dish is {hit_dish} 🥑. If I provide a filtered menu, use it: {filtered_menu}. Current cart: {cart} 🛒. Keep it short, fun, and helpful! Say 'Welcome to CafeSnap! 👋' only on the first response. If an item isn’t on the menu, say 'Sorry, we don’t have that!' and suggest the hit dish. Offer ingredients if asked (e.g., 'Latte has espresso, steamed milk'). Use emojis where fun!"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{user_input}")
])

In [12]:
# Step 9: Build the Chain
chain = RunnableLambda(lambda x: preprocess_input(x)) | prompt | llm
chatbot = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="user_input",
    history_messages_key="chat_history"
)

In [ ]:
# Step 10: Test the Chatbot
def run_chatbot():
    session_id = "user1"  
    print("Start chatting with CafeSnap! Type 'exit' to quit.\n")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Thanks for visiting CafeSnap! ☕")
            break
        
        # Invoke the chatbot with the user input
        response = chatbot.invoke(
            {"user_input": user_input, "session_id": session_id},
            config={"configurable": {"session_id": session_id}}
        )
        print(f"CafeSnap: {response.content}\n")


print("=== WELCOME TO CAFESNAP ===")
session_id = "user1"

# Test 1: Initial greeting and menu
response1 = chatbot.invoke(
    {"user_input": "What’s on the menu?", "session_id": session_id},
    config={"configurable": {"session_id": session_id}}
)
print(f"CafeSnap: {response1.content}\n")

# Test 2: Ordering something
response2 = chatbot.invoke(
    {"user_input": "I want a Latte", "session_id": session_id},
    config={"configurable": {"session_id": session_id}}
)
print(f"CafeSnap: {response2.content}\n")

# Test 3: Checking the cart
response3 = chatbot.invoke(
    {"user_input": "What’s in my cart?", "session_id": session_id},
    config={"configurable": {"session_id": session_id}}
)
print(f"CafeSnap: {response3.content}\n")

run_chatbot()

=== WELCOME TO CAFESNAP ===
CafeSnap: Welcome to CafeSnap! 👋 We've got a delicious menu for you! 🤩 Here's what we've got:

Espresso: $3.00 - A bold, quick caffeine kick! (coffee, quick)
Latte: $4.00 - Smooth and creamy coffee bliss. (coffee, creamy)
Muffin: $2.50 - Sweet blueberry goodness in every bite. (sweet, snack)
Avocado Toast: $5.00 - Trendy, fresh, and oh-so-tasty! (trendy, healthy)

What can I get for you? 🤔

CafeSnap: Added Latte to your cart! 🛒 Anything else? (Cart: Latte)

CafeSnap: Your cart: Latte 🛒. Ready to order more? 😊

Start chatting with CafeSnap! Type 'exit' to quit.



You:  what is under 5$


CafeSnap: 😊 We've got three options under $5:

Espresso: $3.00 - A bold, quick caffeine kick! (coffee, quick)
Muffin: $2.50 - Sweet blueberry goodness in every bite. (sweet, snack)
Latte: $4.00 - Smooth and creamy coffee bliss. (coffee, creamy)

Which one catches your eye? 👀



You:  i want latte


CafeSnap: Added Latte to your cart! 🛒 Anything else? (Cart: Latte, Latte)



...

CafeSnap: Your current order: Latte, Latte 🛒. Ready to finalize? 😊



You:  checkout


CafeSnap: You've checked out! Your total comes to be $8.00. Enjoy your Lattes! ☕️ Thanks for stopping by CafeSnap! 👋



You:  bye


CafeSnap: See you next time at CafeSnap! 👋

